In [7]:
from __future__ import print_function

import torch
import torch.optim as optim

from torch.utils.data.dataset import Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
torch.backends.cudnn.bencmark = True
import torchvision
import torchvision.transforms as transforms

import os,sys,cv2,random,datetime,time,math
import argparse
import numpy as np

from net_s3fdnew import *
from s3fd import *
from bbox import *
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image

In [8]:

class CelebDataset(Dataset):
    """Dataset wrapping images and target labels
    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['Image_Name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['Image_Name']
        self.y_train = self.mlb.fit_transform(tmp_df['Gender'].str.split()).astype(np.float32)
    def __getitem__(self, index):
        img = cv2.imread(self.img_path + self.X_train[index] + self.img_ext)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        #img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        #img = Variable(torch.from_numpy(img).float(),volatile=True)
        
        #if self.transform is not None:
        #    img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [9]:
transformations = transforms.Compose(
    [
     transforms.ToTensor()
     
     #transforms.Normalize(mean=[104,117,123])
     ])

In [10]:
train_data = "index.csv"
img_path = "data/Celeb_Small_Dataset/"
img_ext = ".jpg"
dset = CelebDataset(train_data,img_path,img_ext,transformations)
train_loader = DataLoader(dset,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

In [11]:
def save(model, optimizer, loss, filename, loss_array):
    save_dict = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.data[0],
        'loss_array': loss_array
        }
    
    torch.save(save_dict, filename)

In [12]:
def train_model(model, criterion, optimizer, num_classes, num_epochs = 100):
    loss_array={0:[], 1:[],2:[],3:[]}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0

        for i,(img,label) in enumerate(train_loader):
            img = img.view((1,)+img.shape[1:])
            if use_cuda:
                data, target = Variable(img.cuda()), Variable(torch.Tensor(label).cuda())
            else:
                data, target = Variable(img), Variable(torch.Tensor(label))
            target = target.view(num_classes,1)

            optimizer.zero_grad()
            # would need to add the code here to classify the 
            olist = model(data)
            for p in range(len(olist)):
                olist[p]= F.softmax(olist[p])
                
            for j in range(int(len(olist)/2)):
                ocls,gen = olist[j*2].data.cpu(),olist[j*2+1].data.cpu()
                FB, FC, FH, FW= ocls.size()
                stride= 2**(j+2)
                anchor= stride*4
                for Findex in range(FH*FW):
                    windex,hindex = Findex%FW,Findex//FW
                    axc,ayc = stride/2+windex*stride,stride/2+hindex*stride
                    score = ocls[0,1,hindex,windex]
                    if score<0.05: continue

                    loss = criterion(olist[j*2+1], target)

                    loss.backward(retain_graph=True)
                    optimizer.step()
                    running_loss += loss.data[0]
                    loss_array[j].append(loss.data[0])
                    break
                    
            if i%50==0:
                print("Reached iteration ",i)
                running_loss += loss.data[0]

            
        if epoch % 10 == 0:
            save(model, optimizer, loss, 'faceRecog.saved.model', loss_array)
        print(running_loss)
        with open('loss.csv', 'a') as file:      
            file.writelines(str(running_loss)+ " ")


In [14]:
use_cuda = True
myModel.eval()


s3fd(
  (conv1_1): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d (256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Con

### train

In [ ]:

num_classes = 2
myModel = s3fd(num_classes)
loadedModel = torch.load('s3fd_convert.pth')
newModel = myModel.state_dict()
pretrained_dict = {k: v for k, v in loadedModel.items() if k in newModel}
newModel.update(pretrained_dict)
myModel.load_state_dict(newModel)

In [ ]:
criterion = nn.BCELoss()

for param in myModel.parameters():
    param.requires_grad = False
    
state_dict = myModel.state_dict()

myModel.conv4_3_norm_gender = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)
myModel.l4_3 = nn.Linear(2048,50)
myModel.l4_f = nn.Linear(50,num_classes)
l= loadedModel['conv4_3_norm_mbox_conf.weight']
l[1]=l[0]
state_dict['conv4_3_norm_gender.weight']=l

myModel.conv5_3_norm_gender = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)
myModel.l5_3 = nn.Linear(2048,50)
myModel.l5_f = nn.Linear(50,num_classes)
l= loadedModel['conv5_3_norm_mbox_conf.weight']
l[1]=l[0]
state_dict['conv5_3_norm_gender.weight']=l

myModel.convfc7_norm_gender = nn.Conv2d(1024, 2, kernel_size=3, stride=1, padding=1)
myModel.lfc7 = nn.Linear(288,50)
myModel.lfc7f = nn.Linear(50,num_classes)
l= loadedModel['fc7_mbox_conf.weight']
l[1]=l[0]
state_dict['convfc7_norm_gender.weight']=l

myModel.conv6_2_norm_gender = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)
myModel.l6_2 = nn.Linear(288,50)
myModel.l6_f = nn.Linear(50,num_classes)
l= loadedModel['conv6_2_mbox_conf.weight']
l[1]=l[0]
state_dict['conv6_2_norm_gender.weight']=l

myModel.load_state_dict(state_dict)


optimizer = optim.SGD(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.01, momentum=0.9)
if use_cuda:
    myModel = myModel.cuda()
model_ft = train_model(myModel, criterion, optimizer, num_classes, num_epochs=1000)

### retrain

In [ ]:
criterion = nn.BCELoss()

num_classes = 2
myModel = s3fd(num_classes)
file = torch.load('faceRecog.saved.model')
pretrained_dict = file['model_state_dict']
myModel.load_state_dict(pretrained_dict)

optimizer = optim.SGD(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.01, momentum=0.9)
if use_cuda:
    myModel = myModel.cuda()
model_ft = train_model(myModel, criterion, optimizer, num_classes, num_epochs=1000)

Epoch 0/999
----------


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([1, 2])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1486.5665263533592
Epoch 1/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1486.4904508143663
Epoch 2/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1486.2629038468003
Epoch 3/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1484.5968556702137
Epoch 4/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1489.8100982755423
Epoch 5/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1489.3377259969711
Epoch 6/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
1488.5271205753088
Epoch 7/999
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration

### test

In [ ]:
def transform(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        
        return Variable(img.cuda())
myModel = myModel.cuda()
testImage1 = transform('data/Test/TestCeleb_3/24-FaceId-0.jpg')
testImage2 = transform('data/Test/TestCeleb_4/26-FaceId-0.jpg')
testImage3 = transform('data/Test/TestCeleb_4/27-FaceId-0.jpg')
testImage4 = transform('data/Test/TestCeleb_10/25-FaceId-0.jpg')
testImage5 = transform('data/Test/TestCeleb_10/26-FaceId-0.jpg')
testImage6 = transform('data/Test/TestCeleb_10/24-FaceId-0.jpg')

__,ge2,__,ge3,__,ge4,__,ge5 = myModel(testImage1)
olist = myModel(testImage6)
for p in range(len(olist)):
    olist[p]= F.softmax(olist[p])
                
for j in range(int(len(olist)/2)):
    ocls,gen = olist[j*2].data.cpu(),olist[j*2+1].data.cpu()
    FB, FC, FH, FW= ocls.size()
    stride= 2**(j+2)
    anchor= stride*4
    for Findex in range(FH*FW):
        windex,hindex = Findex%FW,Findex//FW
        axc,ayc = stride/2+windex*stride,stride/2+hindex*stride
        score = ocls[0,1,hindex,windex]
        if score<0.05: continue
        print(gen)
        break

# output2 = myModel(testImage2)
# output3 = myModel(testImage2)
# output4 = myModel(testImage4)
# output5 = myModel(testImage5)
# output6 = myModel(testImage6)
# print("testImage1 - ",output1)
# print("testImage2 - ",output2)
# print("testImage3 - ",output3)
# print("testImage1 - ",output4)
# print("testImage2 - ",output5)
# print("testImage3 - ",output6)

### ignore below

In [ ]:
testImage1 = transform('data/Test/TestCeleb_5/24-FaceId-0.jpg')
output1 = myModel(testImage1)
print("testImage1 - ",output1)

In [2]:
file = torch.load('faceRecog.saved.model')

In [ ]:
with open('loss.csv', 'r') as file:
    x= file.read()

In [6]:
file['model_state_dict'].keys()

loadedModel = torch.load('s3fd_convert.pth')
l=torch.load('s3fd_convert.pth')['conv4_3_norm_mbox_conf.weight']
l[1]=l[0]
l.shape

odict_keys(['conv1_1.weight', 'conv1_1.bias', 'conv1_2.weight', 'conv1_2.bias', 'conv2_1.weight', 'conv2_1.bias', 'conv2_2.weight', 'conv2_2.bias', 'conv3_1.weight', 'conv3_1.bias', 'conv3_2.weight', 'conv3_2.bias', 'conv3_3.weight', 'conv3_3.bias', 'conv4_1.weight', 'conv4_1.bias', 'conv4_2.weight', 'conv4_2.bias', 'conv4_3.weight', 'conv4_3.bias', 'conv5_1.weight', 'conv5_1.bias', 'conv5_2.weight', 'conv5_2.bias', 'conv5_3.weight', 'conv5_3.bias', 'fc6.weight', 'fc6.bias', 'fc7.weight', 'fc7.bias', 'conv6_1.weight', 'conv6_1.bias', 'conv6_2.weight', 'conv6_2.bias', 'conv7_1.weight', 'conv7_1.bias', 'conv7_2.weight', 'conv7_2.bias', 'fc_1.weight', 'fc_1.bias', 'conv3_3_norm.weight', 'conv4_3_norm.weight', 'conv5_3_norm.weight', 'conv3_3_norm_mbox_conf.weight', 'conv3_3_norm_mbox_conf.bias', 'conv3_3_norm_mbox_loc.weight', 'conv3_3_norm_mbox_loc.bias', 'conv4_3_norm_mbox_conf.weight', 'conv4_3_norm_mbox_conf.bias', 'conv4_3_norm_mbox_loc.weight', 'conv4_3_norm_mbox_loc.bias', 'conv4_3_